In [1]:
# Importación librerías
import pandas as pd
import numpy as np
from pycaret.regression import *
from pycaret.time_series import *
from prophet import Prophet
from scipy.stats import uniform
from mango import Tuner
import json
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout,BatchNormalization
import keras_tuner as kt
from tensorflow. keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [8]:
# Lectura de datos
df = pd.read_excel("reporte_diario_campaña_limpio.xlsx").drop(columns="Unnamed: 0")

In [4]:
# Definición error 
def wape(y,y_pred,**kwargs):
    wape = np.sum(np.abs(y - y_pred)) / np.sum(y)
    return wape

In [5]:
# Entrenamiento Pycaret Regresión
def entrenamiento_pycaret_reg(datos_entrenamiento, datos_prueba,target):
    reg = RegressionExperiment()
    reg.setup(datos_entrenamiento, target=target, session_id=47, preprocess=False, verbose=False,
              fold=3,system_log=False,memory=False,data_split_shuffle=False,fold_strategy="timeseries"),
    reg.add_metric('wape', 'WAPE', wape, greater_is_better=False)
    best_reg = reg.compare_models(sort="WAPE", verbose=False, fold = 3)
    best_model_reg = reg.create_model(best_reg,fold=3,verbose = False)
    tuned_reg = reg.tune_model(best_model_reg,verbose=False, fold=3,optimize = "WAPE")
    pred_reg = reg.predict_model(tuned_reg, data=datos_prueba.drop(target, axis=1))
    return pred_reg, best_reg,tuned_reg.get_params()

In [6]:
# Entrenamiento Pycaret Time series
def entrenamiento_pycaret_ts(datos_entrenamiento,datos_prueba,n_periodos,target,fold):
    ts= TSForecastingExperiment()
    ts.setup(datos_entrenamiento, target=target, session_id=47,fh=n_periodos,  verbose=False,fold_strategy="rolling",fold=fold)
    best_ts = ts.compare_models(sort="MAPE", verbose=False)
    best_model_ts = ts.create_model(best_ts,verbose = False)
    tuned_ts = ts.tune_model(best_model_ts,verbose=False,optimize = "MAPE")
    pred_ts = ts.predict_model(tuned_ts, X=datos_prueba.drop(target, axis=1), fh=n_periodos, verbose=False)
    
    return pred_ts, best_ts,tuned_ts.get_params()

In [7]:
# Entrenamiento Prophet
def entrenamiento_prophet(datos, holidays, n_periodos):

    n = len(datos[:-n_periodos])
    n_train = int(n * 0.8)  
    train_df = datos[:n_train]
    val_df = datos[n_train:]
    test_df = datos[-n_periodos:]
    
    param_space = dict(
        growth=['linear', 'flat'],
        n_changepoints=range(0, 55, 5),
        changepoint_range=uniform(0.5, 0.5),
        yearly_seasonality=[True, False],
        weekly_seasonality=[True, False],
        daily_seasonality=[True, False],
        seasonality_mode=['additive', 'multiplicative'],
        seasonality_prior_scale=uniform(5.0, 15.0),
        changepoint_prior_scale=uniform(0.0, 0.1),
        interval_width=uniform(0.2, 0.8),
        uncertainty_samples=[500, 1000, 1500, 2000]
    )
    
    def objective_function(args_list, train_df, val_df):
        results = []
        
        for params in args_list:
            model = Prophet(holidays=holidays, **params)
            model.fit(train_df)
            future = model.make_future_dataframe(periods=len(val_df), freq='D')
            forecast = model.predict(future)
            predictions_tuned = forecast.tail(len(val_df))
            error = wape(val_df['y'], predictions_tuned['yhat'])   
            results.append(error)
            
        return results

    conf_Dict = dict()
    conf_Dict['initial_random'] = 10
    conf_Dict['num_iteration'] = 50

    tuner = Tuner(param_space, lambda x: objective_function(x, train_df, test_df), conf_Dict)
    results = tuner.minimize()

    model = Prophet(holidays=holidays, **results['best_params'])
    model.fit(train_df)
    future = model.make_future_dataframe(periods=len(test_df), freq='D')
    forecast = model.predict(future)
    
    return forecast['yhat'],results['best_params']


In [8]:
# Entrenamiento GRU
def entrenamiento_gru(data,target,linea):
    if target == 'interpolado_real_calls':
        features = ['año', 'mes', "dia", 'dia_semana', 'state_holiday']
    else: 
        features = ['año', 'mes', "dia", 'dia_semana', 'state_holiday','interpolado_real_calls']
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    def train_test_split_by_date(data, date_column, train_end_date, test_start_date, time_step,target):
        data[date_column] = pd.to_datetime(data[date_column])

        adjusted_test_start_date = pd.to_datetime(test_start_date) - pd.Timedelta(days=time_step)

        if target == 'interpolado_real_calls':
            train_data = data[data[date_column] <= train_end_date][['año', 'mes', "dia", 'dia_semana', 'state_holiday',target]].dropna()
            test_data = data[data[date_column] >= adjusted_test_start_date][['año', 'mes', "dia", 'dia_semana', 'state_holiday',target]].dropna()
        else:
            train_data = data[data[date_column] <= train_end_date][['año', 'mes', "dia", 'dia_semana', 'state_holiday',target,'interpolado_real_calls']].dropna()
            test_data = data[data[date_column] >= adjusted_test_start_date][['año', 'mes', "dia", 'dia_semana', 'state_holiday',target,'interpolado_real_calls']].dropna()
        return train_data, test_data
    
    train_data, test_data = train_test_split_by_date(data, 'fecha', '2023-12-31', '2024-01-01',14,target)

    X_train, y_train = train_data[features], train_data[target]
    X_test, y_test = test_data[features], test_data[target]

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

    X_test_scaled = scaler_X.transform(X_test)
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    def create_sequences(X, y, time_steps=10):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:(i + time_steps)])
            ys.append(y[i + time_steps])
        return np.array(Xs), np.array(ys)

    time_steps = 14
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)
    def wape_metric(y_true, y_pred):
        return K.sum(K.abs(y_true - y_pred)) / K.sum(K.abs(y_true))
    def build_robust_gru_model(hp):
        model = Sequential()
        
        # Capas GRU
        for i in range(hp.Int('gru_layers', 1, 3)):
            model.add(GRU(units=hp.Int(f'gru_units_{i}', min_value=64, max_value=512, step=64),
                        return_sequences=(i < hp.Int('gru_layers', 1, 3) - 1),
                        recurrent_dropout=hp.Float(f'recurrent_dropout_{i}', min_value=0.0, max_value=0.3, step=0.1)))
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Capa de normalización por lotes
        model.add(BatchNormalization())

        # Capas densas finales
        for i in range(hp.Int('dense_layers', 1, 3)):
            model.add(Dense(units=hp.Int(f'dense_units_{i}', min_value=32, max_value=256, step=32), activation='relu'))
            model.add(Dropout(rate=hp.Float(f'dense_dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

        model.add(Dense(1))

        # Compilación del modelo
        model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                    loss='mse',  # Seguimos utilizando 'mse' para el entrenamiento
                    metrics=[wape_metric])  # Pero optimizamos usando WAPE
        
        return model

    tuner = kt.RandomSearch(
        build_robust_gru_model,
        objective=kt.Objective('val_wape_metric', direction='min'), 
        max_trials=10,  
        executions_per_trial=1, 
        seed = 47,
        directory=f'gru_tuning_{linea}_{target}',
        project_name=f'gru_tuning_timeseries_{linea}_{target}')

    early_stopping = EarlyStopping(
        monitor='val_wape_metric', 
        patience=7, 
        restore_best_weights=True,
        mode = "min" 
    )
    # Búsqueda de hiperparámetros
    tuner.search(
        X_train_seq, 
        y_train_seq, 
        epochs=20, 
        validation_split=0.2, 
        batch_size=32,
        callbacks=[early_stopping] )
    best_model = tuner.get_best_models(num_models=1)[0]

    best_model.fit(X_train_seq, y_train_seq, epochs=80, validation_split=0.2, batch_size=32)

    y_pred_gru_tuned = best_model.predict(X_test_seq)
    y_pred_gru_tuned_rescaled = scaler_y.inverse_transform(y_pred_gru_tuned)

    return y_pred_gru_tuned_rescaled


In [9]:
def validacion_multiple(dfs, target, linea):

    datos = dfs[dfs["linea"] == linea]
    datos = datos.sort_values(by=["linea", "fecha", "año", "mes", "dia_semana"])
    datos = datos.reset_index(drop=True)

    n_periodos=datos[datos['fecha'].dt.year == 2024].shape[0] # Datos del 2024
    
    datos_red = datos.copy()
    datos_prophet = datos[["fecha", target]].rename(columns={"fecha": 'ds', target: 'y'})
    state_holidays = datos[datos['state_holiday'] == 1]["fecha"].unique()
    state_holidays = pd.DataFrame({'ds': pd.to_datetime(state_holidays), 'holiday': 'state_holiday'})

    if target == "interpolado_real_calls":
        datos = datos[['año', 'mes', "dia", 'dia_semana', target, 'state_holiday']].dropna()
    else: 
        datos = datos[['año', 'mes', "dia", 'dia_semana', target, 'state_holiday',"interpolado_real_calls"]].dropna()
    datos_entrenamiento = datos[:-n_periodos]
    datos_prueba = datos[-n_periodos:]
    news = pd.DataFrame()
    news[target] = datos_prueba[target]
    news["dia_semana"]=datos_prueba["dia_semana"]
    news = news.reset_index(drop=True)
    if linea == "linea_4":
        fold =2
    else:
        fold = 3
    # Entrenamiento con regresión
    pred_reg, mejor_modelo_reg, parametros_reg = entrenamiento_pycaret_reg(datos_entrenamiento, datos_prueba,target)
    news["pred_regr"] = pred_reg["prediction_label"].reset_index(drop=True)

    # Entrenamiento con Time Series
    pred_ts, mejor_modelo_ts, parametros_ts = entrenamiento_pycaret_ts(datos_entrenamiento,datos_prueba,n_periodos,target,fold)
    news["pred_ts"] = pred_ts['y_pred'].reset_index(drop=True)

    # Promedio reg-ts
    news["pred_promedio"] = (news["pred_regr"] + news["pred_ts"]) / 2

    # Entrenamiento con Prophet
    pronostico, mejores_parametros_prophet= entrenamiento_prophet(datos_prophet, state_holidays, n_periodos)
    news["pred_fb"] = pronostico[-n_periodos:].reset_index(drop=True)

    # Entrenamiento con gru
    pred_gru = entrenamiento_gru(datos_red,target,linea)
    news["pred_gru"] = pred_gru
    
    # Eliminar valores negativos
    for i in range(8):
        min_target = datos_entrenamiento.groupby(["dia_semana"]).agg({target: "min"})[target][i]
        news.loc[(news["pred_ts"] < 0) & (news["dia_semana"] == i), "pred_ts"] = min_target
        news.loc[(news["pred_fb"] < 0) & (news["dia_semana"] == i), "pred_fb"] = min_target
        news.loc[(news["pred_regr"] < 0) & (news["dia_semana"] == i), "pred_regr"] = min_target
        news.loc[(news["pred_promedio"] < 0) & (news["dia_semana"] == i), "pred_promedio"] = min_target
        news.loc[(news["pred_gru"] < 0) & (news["dia_semana"] == i), "pred_gru"] = min_target
    

    news["linea"] = linea
    
    # Diccionario final
    dic_final = {
        "pycaret_reg": {"modelo": mejor_modelo_reg,"parametros":parametros_reg},
        "pycaret_ts": {"modelo": mejor_modelo_ts,"parametros":parametros_ts},
        "prophet": mejores_parametros_prophet
    }
    return news, dic_final



# CALLS

In [10]:
lineas = df["linea"].unique()
validacion_calls = []
diccionario_calls = {}

for linea in lineas:
    print(f"Procesando línea: {linea}")
    validacion, diccionario = validacion_multiple(df, "interpolado_real_calls", linea)
    validacion_calls.append(validacion)
    diccionario_calls[linea] = diccionario


Procesando línea: linea_1


Optimization terminated abnormally. Falling back to Newton.


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.


Reloading Tuner from gru_tuning_linea_1_interpolado_real_calls\gru_tuning_timeseries_linea_1_interpolado_real_calls\tuner0.json

Epoch 1/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 14s 76ms/step - loss: 0.2491 - wape_metric: 0.4722 - val_loss: 0.2482 - val_wape_metric: 0.2778
Epoch 2/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.1707 - wape_metric: 0.3767 - val_loss: 0.2209 - val_wape_metric: 0.2651
Epoch 3/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1725 - wape_metric: 0.3753 - val_loss: 0.2379 - val_wape_metric: 0.3147
Epoch 4/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.1624 - wape_metric: 0.3572 - val_loss: 0.2020 - val_wape_metric: 0.2603
Epoch 5/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.2028 - wape_metric: 0.4078 - val_loss: 0.1852 - val_wape_metric: 0.2406
Epoch 6/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.1350 - wape_metric: 0.3463 - val_loss: 0.4798 - val_wape_metric: 0.5174
Epoch 7/80
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.1531 - wape

...

Reloading Tuner from gru_tuning_linea_4_interpolado_real_calls\gru_tuning_timeseries_linea_4_interpolado_real_calls\tuner0.json
Epoch 1/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 246ms/step - loss: 1.1685 - wape_metric: 1.1415 - val_loss: 0.8252 - val_wape_metric: 0.9064
Epoch 2/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 1.3244 - wape_metric: 1.1541 - val_loss: 0.8677 - val_wape_metric: 1.0032
Epoch 3/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 1.1406 - wape_metric: 1.1206 - val_loss: 0.8291 - val_wape_metric: 0.9323
Epoch 4/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 1.1104 - wape_metric: 1.0778 - val_loss: 0.8605 - val_wape_metric: 0.9930
Epoch 5/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 1.2133 - wape_metric: 1.0852 - val_loss: 0.8239 - val_wape_metric: 0.9090
Epoch 6/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 1.0868 - wape_metric: 1.0378 - val_loss: 0.8532 - val_wape_metric: 0.9756
Epoch 7/80
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 1.2441 - wape_metric: 1.087

Optimization terminated abnormally. Falling back to Newton.


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.
Optimization terminated abnormally. Falling back to Newton.


Reloading Tuner from gru_tuning_linea_5_interpolado_real_calls\gru_tuning_timeseries_linea_5_interpolado_real_calls\tuner0.json
Epoch 1/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - loss: 0.2960 - wape_metric: 0.4425 - val_loss: 0.0898 - val_wape_metric: 0.2712
Epoch 2/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.2342 - wape_metric: 0.3854 - val_loss: 0.0994 - val_wape_metric: 0.3200
Epoch 3/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.2687 - wape_metric: 0.4210 - val_loss: 0.1053 - val_wape_metric: 0.3385
Epoch 4/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.1783 - wape_metric: 0.3530 - val_loss: 0.0905 - val_wape_metric: 0.2700
Epoch 5/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.2366 - wape_metric: 0.3867 - val_loss: 0.1002 - val_wape_metric: 0.3161
Epoch 6/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.1836 - wape_metric: 0.3345 - val_loss: 0.2460 - val_wape_metric: 0.6152
Epoch 7/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.2308 - wape_m

In [11]:
validacion_diaria_calls = pd.concat(validacion_calls, ignore_index=True)


In [12]:
validacion_diaria_calls.to_excel("validacion_diaria_calls.xlsx")


In [13]:
def convertir_a_serializable(obj):
    if hasattr(obj, 'get_params'):
        return obj.get_params()
    return str(obj) 

In [14]:
diccionario_serializable = {k: convertir_a_serializable(v) for k, v in diccionario_calls.items()}

In [15]:
with open('diccionario_calls.txt', 'w') as file:
    json.dump(diccionario_serializable, file, indent=4)

## AHT

In [10]:
lineas = df["linea"].unique()
validaciones_aht = []
diccionarios_aht = {}

for linea in lineas:
    print(f"Procesando línea: {linea}")
    validacion_aht, diccionario_aht = validacion_multiple(df, "interpolado_real_aht", linea)
    validaciones_aht.append(validacion_aht)
    diccionarios_aht[linea] = diccionario_aht

Trial 10 Complete [00h 00m 29s]
val_wape_metric: 1.0394723415374756

Best val_wape_metric So Far: 0.9572116732597351
Total elapsed time: 00h 14m 09s
Epoch 1/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 137ms/step - loss: 1.6078 - wape_metric: 1.1963 - val_loss: 0.9576 - val_wape_metric: 1.1079
Epoch 2/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - loss: 1.1319 - wape_metric: 1.0354 - val_loss: 1.0174 - val_wape_metric: 1.1778
Epoch 3/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - loss: 0.9481 - wape_metric: 0.9564 - val_loss: 0.8152 - val_wape_metric: 1.0209
Epoch 4/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - loss: 0.7341 - wape_metric: 0.8098 - val_loss: 0.9821 - val_wape_metric: 1.1378
Epoch 5/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.8116 - wape_metric: 0.8195 - val_loss: 0.9212 - val_wape_metric: 1.0989
Epoch 6/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.6163 - wape_metric: 0.7795 - val_loss: 1.0737 - val_wape_metric: 1.2330
Epoch 7/80
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/s

In [11]:
validacion_diaria_aht = pd.concat(validaciones_aht, ignore_index=True)

In [14]:
diccionario_serializable_aht = {k: convertir_a_serializable(v) for k, v in diccionarios_aht.items()}

In [15]:
validacion_diaria_aht.to_excel("validacion_diaria_aht.xlsx")
with open('diccionario_aht.txt', 'w') as file:
    json.dump(diccionario_serializable_aht, file, indent=4)